<a href="https://colab.research.google.com/github/shinb-bong/TIL/blob/main/DFS_BFS_%EA%B8%B0%EC%B6%9C_%EB%AC%B8%EC%A0%9C%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

특정 거리의 도시 찾기
----

풀이: N,M의 주어진 수가 매우 크기 때문에 완전 탐색은 별로인것 같다. 또한 *** 모든 간선의 길이가 1이라는 점** 에서 BFS 진행

In [ ]:
from collections import deque

n,m,k,x = map(int,input().split())

result =  [[0] for _ in range(n+1)]
for i in range(m):
  a,b = map(int,input().split())
  result[a].append(b)

distance = [-1]*(n+1) # 모든 최단 거리 테이블 초기화
distance[x] = 0 # 처음 출발하는 자기 자신은 0이라고 했으니

q = deque([x])

while q:
  now = q.popleft()
  # 현재 도시에서 이동 할 수 있는 모든 도시 확인
  for next in result[now]:
    if distance[next] == -1: # 아직 방문 되지 않은거라면
      distance[next] = distance[now]+1
      q.append(next)

check = False
for i in range(1,n+1):
  if distance[i] == k:
    print(i)
    check = True

if check == False:
  print("-1")







4 4 2 1
1 2
1 3
2 3
2 4
4


연구소 
---

풀이: 어디든 놓는걸 계산했을때 최대 64C3 인데 엄청 작은 수니깐 하나씩 DFS/BFS 확인해가면서 벽 세울 곳을 계산해도 괜찮다

이 문제는 완전탐색을 진행하면서 DFS/BFS 를 진행하는 문제라는데 두가지 방법 다 공부해보려고한다.

또한 이렇게 "바이러스가 사방으로 나간다" 처럼 주체가 방향성을 띄고있을때는 **동서남북 방향 벡터**를 사용해서 접근하는 것이 좋을 것 같다.

BFS 참고: https://hongcoding.tistory.com/m/130

In [ ]:
#BFS
from collections import deque
import copy

# 바이러스가 있는 자리 확인하는것 
def bfs():
  queue = deque()
  tmp = copy.deepcopy(graph)

  # 바이러스를 하나씩 확인하여 있는 자리 파악
  for i in range(n):
    for j in range(m):
      if tmp[i][j] == 2:
        queue.append((i,j))

  # 바이러스 확산 계산하는거 
  while queue:
    x,y =queue.popleft()
    for i in range(4):
      nx = x + dx[i]
      ny = y + dy[i]

      if nx<0 or nx>=n or ny <0 or ny >=m:
        continue
      if tmp[nx][ny] == 0:
        tmp[nx][ny] = 2
        queue.append((nx,ny))

    global answer 
    cnt = 0

    for i in range(n):
      cnt += tmp[i].count(0)
    
    answer = max(answer,cnt)

def makeWall(cnt):
    if cnt == 3:
        bfs()
        return

# 벽을 하나씩 다 넣어 보면서 확인하는 것 
    for i in range(n):
        for j in range(m):
            if graph[i][j] == 0:
                graph[i][j] = 1
                makeWall(cnt+1)
                graph[i][j] = 0

n, m = map(int, input().split())
graph = []
dx = [0, 0, 1, -1]
dy = [1, -1, 0, 0]

for i in range(n):
    graph.append(list(map(int, input().split())))

answer = 0
makeWall(0)
print(answer)

결국 두 방법다 마지막에 바이러스 접근 방식이 다른것 뿐이지 핵심은 두 방법다 조합을 하였을때 최대 발생 할 수 있는 연산 수가 작기때문에 일일히 다 한번씩 세워보고 계산하고 아니면 다시 다음걸로 넘어가고 하는 모든 완전 탐색이 가능하다는 점이였다.

In [ ]:
# DFS 사용
# DFS는 재귀적함수를 자주 이용한다.

n, m = map(int, input().split())
data = [] # 초기 맵

# 사방 이동 방향 벡터
dx = [0, 0, 1, -1]
dy = [1, -1, 0, 0]
temp = [[0]*m for _ in range(n)] # 벽 세운뒤 맵

for i in range(n):
    data.append(list(map(int, input().split())))


result = 0 

# DFS로 각 바이러스가 사방으로 퍼지게만들기
def virus(x,y):
  for i in range(4):
    nx = x+ dx[i]
    ny = y+ dy[i]

    # 상하좌우 중에서 바이러스가 퍼질수 있는 경우
    if nx >= 0 and nx<n and ny>= 0 and ny <m:
      if temp[nx][ny] == 0:
        # 해당 위치에 바이러스 배치, 재귀
        temp[nx][ny] == 2
        virus(nx,ny)

def safe():
  score  =0
  for i in range(n):
    for j in range(m):
      if temp[i][j] == 0:
        score += 1
  return score


def dfs(cnt):
  global result
  # 울타리가 3개 설치 된경우
  if cnt == 3:
    for i in range(n):
      for j in range(m):
        # 벽 세워진 그래프로 복사(위는 deepcopy를 사용)
        temp[i][j] =data[i][j]
    # 각자리에서 전파 시작
    for i in range(n):
      for j in range(m):
        if temp[i][j] == 2:
          virus(i,j)
    # 안전 구역 최대값 구함
    result = max(result,safe())
    return 

  for i in range(n):
    for j in range(m):
      if data[i][j] == 0:
        # 하나씩 설치해보고 풀었다가 하면서 최대값 찾기
        data[i][j] = 1
        cnt += 1
        dfs(cnt)
        data[i][j] = 0
        cnt -= 1
# 0개로 시작
dfs(0)

print(result)

경쟁적 전염 
---

망한 풀이:  아래는 망한 코드이다.
+ 방향 벡터로 접근을 하여 만약 비어있거나 해당 칸이 현재 수보다 클 경우를 덮어쓰기를 하려했지만 아마 시간 복잡도도 너무 크고 실행을 했을때 올바른 답이 안나오는 걸로 보아 오류가 있는듯하다.

정확한 풀이: "낮은 번호"부터라는 점을 유의해서 보자. 낮은 번호라 함은 당연하게 큐를 떠올려야한다. BFS로 접근하는것이 좋을 것 같다.

In [6]:
# 망한 풀이
n,m = map(int,input().split())

graph = []

for i in range(n):
  graph.append(list(map(int,input().split())))

s,x,y = map(int,input().split())

# 상하 좌우 방향 벡터 설정 (동,서,남,북)
dx = [0,0,-1,1]
dy = [1,-1,0,0]

def dfs(graph):
  for i in range(n):
    for j in range(n):
      # 번호 확인
      now = graph[i][j]
      # 상하좌우 이동
      for k in range(4):
        nx = dx[k] + x
        ny = dx[k] + y

        if nx<0 or ny<0  or nx>=n or ny>= n:
          continue

        if graph[nx][ny] > now or graph[nx][ny] == 0 :
          graph[nx][ny] = now
        
for i in range(s):
  dfs(graph)

print(graph[y-1][x-1])




3 3
1 0 2
0 0 0
3 0 0
2 3 2
0


+ 놓친 포인트: s를 계속 해서 저장할 필요가 없고 한번 전파된 바이러스는 큐에서 나가니 s 를 데이터에 같이 넣어서 확인을 해주는 테크닉을 사용했다.

In [ ]:
# 인접리스트 구현 방식 BFS
from collections import deque

n,k = map(int,input().split())

graph = [] # 전체 보드 정보
data = [] # 바이러스에 대한 정보를 담는 리스트

# 위 두 리스트에 정보 입력 시작
for i in range(n):
  # 보드 정보를 한 줄 단위로 입력
  graph.append(list(map(int,input().split())))

  for j in range(n):
    # 해당 위치에 바이러스 존재하는 경우
    if graph[i][j] != 0:
      # (바이러스 종류, 시간, 위치, X, Y) 삽입 , 우선 순위 큐를 사용할꺼니깐 종류를 맨 앞에서 위치
      data.append((graph[i][j],0,i,j))

# 정렬을 진행해서 큐로 옮김 (낮은것 부터 진행하는걸 구현)
data.sort()

# 우선 순위 큐
q = deque(data)

# 목표 받아오기
t_s,t_x,t_y = map(int,input().split())

# 상하 좌우 방향 벡터 설정 (동,서,남,북)
dx = [0,0,-1,1]
dy = [1,-1,0,0]

# BFS

while  q:
  virus,s,x,y = q.popleft()

  # 정확히 s가 지나거나 큐가 빌때까지 반복
  if s == t_s:
      break
  for i in range(4):
    nx = x+dx[i]
    ny = y+dy[i]
    # 해당 위치 이동가능
    if 0<= nx and nx<n and 0<=ny and ny< n :
      # 아직 방문하지 않은 노드라면?
      if graph[nx][ny] == 0:
        graph[nx][ny] = virus
        # 증가된 시간 넣어주기
        q.append((virus,s+1,nx,ny))

print(graph[t_s-1][t_y-1])
  

괄호 변환
---

풀이: 재귀함수 사용 및 문제 그대로 따라가면 되는 유형이였다. 근데 나는 처음에 고민을 많이해서 조금 시간이 더 걸린감이 있던것 같다

In [ ]:
# 계속 작게작게 짜르는 문제

# 균형 잡힌 문자열 확인
def balance(p):
    cnt = 0 
    for i in range(len(p)):
        if p[i] == "(":
            cnt += 1
        else:
            cnt -= 1
        # 첫 균형 잡힌 인덱스 반환
        if cnt == 0:
            return i
    

# 올바른 문자열인지 확인하는 메소드
def check(p):
    cnt = 0 
    for i in p:
        if i == "(":
            cnt +=1
        else:
            if cnt == 0:
                return False
            cnt -=1
    return True # 쌍이 맞는경우

def solution(p):
    
    answer = ''
    if p == '':
        return answer
    # 첫 균형잡힌 리스트 분해
    index = balance(p)
    u = p[:index+1]
    v = p[index+1:]
    
    # 만약 u 가 올바른 문자열이라면
    if check(u):
        # 재귀 진행하여 v확인하고 합체
        answer = u + solution(v)
        
    # 아니라면
    else:
        answer ='('
        
        # 4-2 문장 작성
        answer += solution(v)
        answer += ')'
        
        u = list(u[1:-1]) # 4-4 처음과 마지막 문자 제거
        
        for i in range(len(u)):
            if u[i] == '(':
                u[i] = ')'
            else:
                u[i] = '('
        answer += "".join(u)               
    return answer
    

연산자 끼워 넣기
---

풀이:  완전 탐색을 해야할때 ( 다 검사해야할때) 해당 연산을 줄였다가 재귀하고 아닌게 판정이나면 다시 해당 연산을 더해주는 그런 테크닉이 자주 쓰이는것 같다.

In [9]:
n = int(input())

data = list(map(int,input().split()))

cal = list(map(int,input().split()))

# 최소 최대 초기화 
min_value = 1e9
max_value = -1e9

def dfs(i,now):

  # 연산들 들고오기
  global cal,min_value,max_value
  # 연산들 모두 소진
  if i ==n:
    min_value = min(min_value,now)
    max_value = max(max_value,now)
  else:
    # 각 연산자에 대해서 재귀적으로 수행 풀이에서 적은 부분
    if cal[0] > 0:
      cal[0] -=1
      dfs(i+1,now+data[i])
      cal[0] +=1
    if cal[1] > 0:
      cal[1] -=1
      dfs(i+1,now-data[i])
      cal[1] +=1
    if cal[2] > 0:
      cal[2] -=1
      dfs(i+1,now*data[i])
      cal[2] +=1
    if cal[3] > 0:
      cal[3] -=1
      dfs(i+1,int(now/data[i]))
      cal[3] +=1

# 첫 연산이라고 표기해주고 호출
dfs(1,data[0])

# 최대 최소 출력
print(max_value)
print(min_value)

2
5 6
0 0 1 0
30
30


감시피하기
---


인구 이동
---

BFS를 사용하여 인구 이동 처리 and 연합 index를 사용하여 연합끼리 묶어주면서 진행하는 방식을 사용하였다. 큐를 사용하는거에 조금 더 익숙해질 필요가 있다.

In [ ]:
from collections import deque
n,l,r = map(int,input().split())

city = []

# 도시 입력
for i in range(n):
  city.append(list(map(int,input().split())))

# 도시의 방향벡터
dx = [1,0,-1,0]
dy = [1,-1,0,0]

result = 0

# 특정 위체에서 출발하여 모든 연합을 체크 한뒤에 데이터 갱신
def bfs(x,y,index):
  # (x,y)의 위치와 연결된 나라의 정보를 담는다.
  united = []
  united.append((x,y))
  # BFS 자료구조
  q = deque()
  q.append((x,y))

  union[x][y] = index # 현재 연합번호를 할당
  summary = city[x][y] # 현재 연합의 전체인구수

  count = 1 # 현재 연합의 국가수
  # 큐가 빌때까지
  while q:
    x,y = q.popleft()
    # 현재 위치에서 4가지 방향확인
    for i in range(4):
      nx = x + dx[i]
      ny = y + dy[i]
      # 바로 옆에있는 나라확인(외부가 아니고 연합이 되지 않았다면)
      if 0 <= nx <n and 0<=ny <n and union[nx][ny] == -1:
        # 옆에 있는 나라와 인구 차이가 L 이상,R 이하
        if l <= abs(city[nx][ny]-city[x][y]) <=r :
          q.append((nx,ny))
          # 연합에 추가
          union[nx][ny] = index
          summary += graph[nx][ny]
          count += 1
          united.append((nx,ny))
  for i, j in united:
    city[i][j] = summary //count # 문제에 적혀있음
  return count

total_count = 0

# 더 이상 이동이 불가 할때까지 이동
while True: 
  union = [[-1]*n for _ in range(n)]
  index = 0
  for i in range(n):
    for j in range(n):
      if union[i][j] == -1: # 아직나라가 처리가 안되었다면
        bfs(i,j,index) # 연합국가 처리
        index += 1

  #모든 인구 이동이 끝나면
  if index  == n *n :
    break
  
  total_count += 1

# 인구 이동 횟수 출력
print(total_count)
    
